In [7]:
dotnet build ../examples/userstorygenerator/UserStoryGenerator
dotnet build ../examples/batcheval/library

MSBuild version 17.8.3+195e7f5a3 for .NET
  Determining projects to restore...
  All projects are up-to-date for restore.
  UserStoryGenerator -> C:\Users\mapach\repos\dotnet-llm-eval-samples\examples\userstorygenerator\UserStoryGenerator\bin\Debug\net8.0\UserStoryGenerator.dll

Build succeeded.
    0 Warning(s)
    0 Error(s)

Time Elapsed 00:00:02.29
MSBuild version 17.8.3+195e7f5a3 for .NET
  Determining projects to restore...
  Restored C:\Users\mapach\repos\dotnet-llm-eval-samples\examples\batcheval\library\BatchEvalCore.csproj (in 419 ms).
  BatchEvalCore -> C:\Users\mapach\repos\dotnet-llm-eval-samples\examples\batcheval\library\bin\Debug\net8.0\BatchEvalCore.dll

Build succeeded.
    0 Warning(s)
    0 Error(s)

Time Elapsed 00:00:03.27


In [28]:
#i "nuget:https://api.nuget.org/v3/index.json" 
#r "nuget:Microsoft.SemanticKernel, 1.0.1"
#r "nuget:OpenTelemetry.Exporter.Console, 1.7.0"
#r "../examples/userstorygenerator/UserStoryGenerator/bin/Debug/net8.0/UserStoryGenerator.dll"
#r "../examples/batcheval/library/bin/Debug/net8.0/BatchEvalCore.dll"

using Microsoft.SemanticKernel;
using UserStoryGenerator;
using BatchEval.Core;

Restore sources https://api.nuget.org/v3/index.json Installed Packages Microsoft.SemanticKernel, 1.0.1 OpenTelemetry.Exporter.Console, 1.7.0

In [10]:
var builder = Kernel.CreateBuilder();

builder.AddAzureOpenAIChatCompletion(
        "gpt-4",
        "<openAiEndpoint>",
        "<openAiKey>");

var kernel = builder.Build();

In [11]:
internal struct UserInput
{
    public string Description { get; set; }
    public string ProjectContext { get; set; }
    public string Persona { get; set; }
}

In [25]:
var dataFilePath = "assets/data.jsonl";
var preview = await BatchEval.Core.JsonlUtils.Preview<UserInput>(dataFilePath);
preview

index value 0 Submission#9+UserInput Description Learn to swim ProjectContext A pond Persona Duck 1 Submission#9+UserInput Description Build a sandcastle ProjectContext At the beach Persona Child 2 Submission#9+UserInput Description Write a novel ProjectContext In a quiet library Persona Author 3 Submission#9+UserInput Description Create a new recipe ProjectContext In the kitchen Persona Chef 4 Submission#9+UserInput Description Solve a complex math problem ProjectContext In a classroom Persona Student 5 Submission#9+UserInput Description Design a user interface ProjectContext At a tech company Persona Designer 6 Submission#9+UserInput Description Plant a garden ProjectContext In the backyard Persona Gardener 7 Submission#9+UserInput Description Train for a marathon ProjectContext On the running track Persona Runner 8 Submission#9+UserInput Description Paint a masterpiece ProjectContext In an art studio Persona Artist 9 Submission#9+UserInput Description Cook a gourmet meal ProjectContext In a fancy restaurant Persona Chef 10 Submission#9+UserInput Description Teach a programming language ProjectContext In a coding bootcamp Persona Instructor 11 Submission#9+UserInput Description Plan a surprise party ProjectContext For a friend's birthday Persona Party Planner 12 Submission#9+UserInput Description Explore a new hiking trail ProjectContext In the mountains Persona Hiker 13 Submission#9+UserInput Description Build a treehouse ProjectContext In the backyard Persona DIY Enthusiast 14 Submission#9+UserInput Description Learn a new language ProjectContext In a language school Persona Language Learner 15 Submission#9+UserInput Description Develop a mobile app ProjectContext In a software company Persona Developer 16 Submission#9+UserInput Description Organize a community cleanup ProjectContext In a local neighborhood Persona Community Organizer 17 Submission#9+UserInput Description Create a budget plan ProjectContext For a small business Persona Accountant 18 Submission#9+UserInput Description Play a musical instrument ProjectContext In a concert hall Persona Musician 19 Submission#9+UserInput Description Write a research paper ProjectContext In a university Persona Researcher (38 more)

In [30]:
internal class UserStoryCreator : BatchEval.Core.IInputProcessor<UserInput>
{
    private readonly UserStorySkill userStoryGenerator;

    public UserStoryCreator(Kernel kernel)
    {
        this.userStoryGenerator = UserStorySkill.Create(kernel);
    }

    public async Task<ModelOutput> Process(UserInput userInput)
    {
        var userStory = await userStoryGenerator.GetUserStory(
            userInput.Description,
            userInput.ProjectContext,
            userInput.Persona);

        return new ModelOutput() {
            Input = $"Generate a user story for {userInput.Persona} so it can {userInput.Description}",
            Output = $"{userStory!.Title} - {userStory!.Description}"
        };
    }
}

In [31]:
//var kernelFunctions = kernel.CreatePluginFromPromptDirectory("Prompts");

var batchEval = new BatchEval<UserInput>();

batchEval
    //.AddEvaluator(new PromptScoreEval("coherence", kernel, kernelFunctions["coherence"]))
    .AddEvaluator(new LenghtEval());

await batchEval
    .WithInputProcessor(new UserStoryCreator(kernel))
    .WithJsonl(dataFilePath)
    .Run();

Q: Generate a user story for Duck so it can Learn to swim
A: Learn to Swim in the Pond - As a Duck, I want to learn to swim in the pond, so I can move around freely and find food
E: length S: 117
Resource associated with Metric:
    telemetry.sdk.name: opentelemetry
    telemetry.sdk.language: dotnet
    telemetry.sdk.version: 1.7.0
    service.name: unknown_service:dotnet

Metric Name: prompt.counter, Meter: AzDoCopilotTests
(2024-01-08T22:45:48.0733088Z, 2024-01-08T22:45:58.0702404Z] LongSum
Value: 1

Metric Name: length.score, Meter: AzDoCopilotTests
(2024-01-08T22:45:48.0745682Z, 2024-01-08T22:45:58.0704431Z] Histogram
Value: Sum: 117 Count: 1 Min: 117 Max: 117 
(-Infinity,0]:0
(0,5]:0
(5,10]:0
(10,25]:0
(25,50]:0
(50,75]:0
(75,100]:0
(100,250]:1
(250,500]:0
(500,750]:0
(750,1000]:0
(1000,2500]:0
(2500,5000]:0
(5000,7500]:0
(7500,10000]:0
(10000,+Infinity]:0



Error: Command cancelled.